In [ ]:
import xarray as xr
import altair as alt
import pandas as pd
import datetime
import glob
import sys
sys.path.append('../')
import sosutils
import seaborn as sns
import numpy as np

alt.data_transformers.disable_max_rows()

In [ ]:
files = glob.glob("/Users/elischwat/Downloads/isfs_*.nc")
files = sorted(files)
files

# Create TKE dataframe

In [ ]:
# all variables (covariances) used in calculation of TKE
TKE_VARIABLE_NAMES = [
    ## central tower
    'u_u__1m_c', 'v_v__1m_c', 'w_w__1m_c',
    'u_u__2m_c', 'v_v__2m_c', 'w_w__2m_c',
    'u_u__3m_c', 'v_v__3m_c', 'w_w__3m_c',
    'u_u__5m_c', 'v_v__5m_c', 'w_w__5m_c',
    'u_u__10m_c', 'v_v__10m_c', 'w_w__10m_c',
    'u_u__15m_c', 'v_v__15m_c', 'w_w__15m_c',
    'u_u__20m_c', 'v_v__20m_c', 'w_w__20m_c',
    ## other towers
    'u_u__1m_ue', 'v_v__1m_ue', 'w_w__1m_ue',
    'u_u__3m_ue', 'v_v__3m_ue', 'w_w__3m_ue',
    'u_u__10m_ue', 'v_v__10m_ue', 'w_w__10m_ue',

    'u_u__1m_uw', 'v_v__1m_uw', 'w_w__1m_uw',
    'u_u__3m_uw', 'v_v__3m_uw', 'w_w__3m_uw',
    'u_u__10m_uw', 'v_v__10m_uw', 'w_w__10m_uw',

    'u_u__1m_d', 'v_v__1m_d', 'w_w__1m_d',
    'u_u__3m_d', 'v_v__3m_d', 'w_w__3m_d',
    'u_u__10m_d', 'v_v__10m_d', 'w_w__10m_d',
]

# Convert dataset to dataframe
tke_df = sosutils.open_datasets_as_dataframe(files, variables=TKE_VARIABLE_NAMES)

# Parse height and tower information from column names so measurement height and tower is a column
tke_df = tke_df.melt(id_vars='time', value_vars=TKE_VARIABLE_NAMES)
tke_df['height'] = tke_df['variable'].apply(sosutils.height_from_variable_name)
tke_df['tower'] = tke_df['variable'].apply(sosutils.tower_from_variable_name)
tke_df['measurement'] = tke_df['variable'].apply(sosutils.measurement_from_variable_name)
tke_df = tke_df.set_index(['time'])

# Calculate TKE
tke_df = tke_df.groupby(['time', 'height', 'tower']).sum(numeric_only=True).reset_index()
tke_df['value'] = 0.5*tke_df['value']

## Create height & time bounds for 2D colormap plotting of TKE

In [ ]:
def height_to_bounds(h):
    if h == 1:
        return (0,1)
    if h == 2:
        return (1,2)
    elif h == 3:
        return (2, 4)
    elif h == 5:
        return (4, 7)
    elif h == 10:
        return (7, 13)
    elif h == 15:
        return (13, 17)
    elif h == 20:
        return (17, 23)
tke_df['height_bounds'] = tke_df['height'].apply(height_to_bounds)
tke_df['height1'] = tke_df['height_bounds'].apply(lambda x: x[0])
tke_df['height2'] = tke_df['height_bounds'].apply(lambda x: x[1])



tke_df['time2'] = tke_df['time'].apply(lambda t: t + datetime.timedelta(minutes = 5))

# Examine TKE

### Distribution of values

In [ ]:
ax = sns.distplot(
    np.log10([f for f in tke_df['value'] if f != 0]),
    axlabel='LOG10[TKE (m^2/s^2)]', 
    norm_hist=True
)

### Characteristic vertical profiles at 1am amd 1pm

In [ ]:
plot = None
for i in range(1, 6):
    local_src = tke_df[tke_df['time'].dt.day == i]
    local_src = local_src[local_src['time'].dt.hour == 1]
    local_src = local_src.groupby(['height', 'tower']).mean().reset_index()
    new_chart = alt.Chart(local_src).mark_line().encode(
            alt.X('value:Q', title='TKE (m^2/s^2)', sort='y', scale=alt.Scale(zero=True)),
            alt.Y('height', title='Height (m)'),
            alt.Color('tower:N')
        ).properties(width=100, title=f'1-2am on November {i}')
    if i == 1:
        plot = new_chart
    else:
        plot = plot | new_chart
plot

In [ ]:
plot = None
for i in range(1, 6):
    local_src = tke_df[tke_df['time'].dt.day == i]
    local_src = local_src[local_src['time'].dt.hour == 13]
    local_src = local_src.groupby(['height', 'tower']).mean().reset_index()
    new_chart = alt.Chart(local_src).mark_line().encode(
            alt.X('value:Q', title='TKE (m^2/s^2)', sort='y', scale=alt.Scale(zero=True)),
            alt.Y('height', title='Height (m)'),
            alt.Color('tower:N')
        ).properties(width=100, title=f'1-2pm on November {i}')
    if i == 1:
        plot = new_chart
    else:
        plot = plot | new_chart
plot

### Temporal evolution of vertical profile

In [ ]:
local_src = tke_df.copy()
local_src['time'] = local_src['time'] - datetime.timedelta(hours = 6)
local_src['time2'] = local_src['time2'] - datetime.timedelta(hours = 6)
plot_tke_tower_c = alt.Chart(local_src).transform_filter(
    alt.datum.tower=='c'
).mark_rect().encode(
    alt.X('time:T', title='Time'),
    alt.X2('time2:T'),
    alt.Y('height1:Q', title='Height (m)'),
    alt.Y2('height2:Q'),
    alt.Color('value:Q', scale=alt.Scale(domain=[0.01,10], type='log'), title='TKE'),
    alt.Facet('tower:N', columns=1)
).properties(width=1400)

plot_tke_tower_c

# Create covariance dataframe

In [ ]:
COVARIANCE_VARIABLE_NAMES = ['u_u__5m_c', 'v_v__5m_c', 'w_w__5m_c']
variances_df = sosutils.open_datasets_as_dataframe(files, variables=COVARIANCE_VARIABLE_NAMES)
variances_df = variances_df.melt(id_vars='time', value_vars=COVARIANCE_VARIABLE_NAMES)
variances_df['height'] = variances_df['variable'].apply(sosutils.height_from_variable_name)
variances_df['tower'] = variances_df['variable'].apply(sosutils.tower_from_variable_name)
variances_df['measurement'] = variances_df['variable'].apply(sosutils.measurement_from_variable_name)

# Examine Covariances

### Temporal evolution @ 5 meters, tower C

In [ ]:
local_src = variances_df[variances_df['time'] < datetime.datetime(2022, 11, 7)]
local_src = local_src[local_src['time'] > datetime.datetime(2022, 11, 3)]
alt.Chart(local_src).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('variable:N')
).properties(width=1300)

# Create wind speed and direction dataframe

In [ ]:
WIND_VARIABLE_NAMES = [    
    'spd_1m_c', 'dir_1m_c',
    'spd_2m_c', 'dir_2m_c',
    'spd_3m_c', 'dir_3m_c',
    'spd_5m_c', 'dir_5m_c',
    'spd_10m_c', 'dir_10m_c',
    'spd_15m_c', 'dir_15m_c',
    'spd_20m_c', 'dir_20m_c',
]

wind_df = sosutils.open_datasets_as_dataframe(files, variables=WIND_VARIABLE_NAMES)

wind_df = wind_df.melt(id_vars='time', value_vars=WIND_VARIABLE_NAMES)
wind_df['height'] = wind_df['variable'].apply(sosutils.height_from_variable_name)
wind_df['tower'] = wind_df['variable'].apply(sosutils.tower_from_variable_name)
wind_df['measurement'] = wind_df['variable'].apply(sosutils.measurement_from_variable_name)

### Pivot table to have wind speed and direction as different columns

In [ ]:
wind_df = wind_df.pivot_table(
   values = ['value'],
   index= ['time', 'height', 'tower'],
   columns= ['measurement']
)
wind_df.columns = wind_df.columns.get_level_values(1)
wind_df = wind_df.reset_index()
wind_df = wind_df.sort_values(['time', 'height', 'tower'])
wind_df.columns.name = None

### Resample by 30 minute intervals

In [ ]:
wind_30min_df = wind_df.set_index('time')
grouper = wind_30min_df.groupby([pd.Grouper(freq='30T'), 'height', 'tower'])


result = grouper.mean()

# result = grouper[['wind direction', 'wind speed']].mean().unstack()
result.columns = result.columns.get_level_values(0)
wind_30min_df = result.reset_index()

### Resample by 60 minute intervals

In [ ]:
wind_60min_df = wind_df.set_index('time')
grouper = wind_60min_df.groupby([pd.Grouper(freq='60T'), 'height', 'tower'])


result = grouper.mean()

# result = grouper[['wind direction', 'wind speed']].mean().unstack()
result.columns = result.columns.get_level_values(0)
wind_60min_df = result.reset_index()

# Examine wind speed and direction

In [ ]:
plot_windspeed_tower_c = alt.Chart(wind_30min_df).transform_filter(
    alt.datum.height == 5
).mark_point().encode(
    alt.X('time:T'),
    alt.Y('wind speed:Q'),
    alt.Color('height:N')
).properties(width=1400)

plot_tke_tower_c & plot_windspeed_tower_c

In [ ]:
plot_windspeed_tower_c = alt.Chart(wind_30min_df).transform_filter(
    alt.datum.height == 5
).mark_point().encode(
    alt.X('time:T'),
    alt.Y('wind speed:Q'),
    alt.Color('height:N')
).properties(width=1400)

plot_tke_tower_c & plot_windspeed_tower_c

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
local_src[local_src['time'].dt.day==2].head(100)

In [ ]:
local_src = wind_60min_df.copy()
local_src['time'] = local_src['time'] - datetime.timedelta(hours = 6)
local_src['day'] = local_src['time'].dt.date.apply(lambda date: datetime.datetime.combine(date, datetime.time()))
local_src['hour'] = local_src['time'].dt.hour

local_src = local_src[local_src['time'] < datetime.datetime(2022, 11, 4)]
alt.Chart(local_src).mark_point(shape="wedge", filled=True).encode(
    alt.X('hour:Q'),
    alt.Y('height:Q'),
    # color=alt.Color(
    #     "wind direction", scale=alt.Scale(domain=[0, 360], scheme="rainbow"), legend=None
    # ),
    angle=alt.Angle("wind direction", scale=alt.Scale(domain=[0, 360], range=[180, 540])),
    size=alt.Size("wind speed", scale=alt.Scale(rangeMax=5000), title='Wind Speed (m/s)'),
).properties(
    width=1000,
    height=200
).facet(
    row='day:T'
)

### Animated video of 5 minute wind interval

In [ ]:
src = wind_30min_df.query("tower == 'c'")

#### For each time interval, add a 0 wind speed at 0 height with direction 0

In [ ]:
for time in src.time.unique():
    src = src.append(
        {
            'height': 0,
            'time': time,
            'wind speed': 0
        },
        ignore_index=True
    )

In [ ]:
dataset = src.sort_values(['time', 'height']).copy()

dt = 5 # minutes
tfinal = 24*60 # 24 hours
x0 = 0

dataset_chunk_size = 8 # number of measurements to take at a time  from the dataframe

In [ ]:
i = 1

In [ ]:
dataset[ i*dataset_chunk_size : (i+1)*dataset_chunk_size]

In [ ]:
dataset.sort_values('wind speed').tail(100)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

def animate(i):
    ax.cla() # clear the previous image
    chunk = dataset[ i*dataset_chunk_size : (i+1)*dataset_chunk_size].sort_values('height')
    ax.plot(chunk['wind speed'], chunk['height'], linewidth=2) # plot the line
    ax.set_xlim([0, 12])
    ax.set_ylim(0, 20)
    ax.annotate(str(chunk.iloc[0].time), xy=(0, 18))
    hr = chunk.iloc[0].time.hour 
    if hr < 6 or hr > 18:
        ax.patch.set_facecolor('lightgrey')
    else:
        ax.patch.set_facecolor('white')


anim = animation.FuncAnimation(fig, animate, frames = range(0, int(tfinal / dt)), blit = False, interval=50)
anim.save('test.gif')

plt.show()

# TKE Budget

## Simplified TKE Equation

Assumptions for simplified TKE equation:
* Coordinate system  is aligned with the mean wind so that terms involving v are zero, 
* Applied over a flat, homogeneous area with no subsidence so that terms involving (∂/∂x), (∂/∂y) and w are also zero


$$
\frac{\partial \overline{e}}{\partial t} = g \frac{\overline{w'\theta_v'}}{\overline{\theta_v}} - \frac{1}{\rho_a}\frac{\partial \overline{w'P'}}{\partial z} - \overline{u'w'} \frac{\partial \overline{u}}{\partial z} - \frac{\partial \overline{w' e}}{\partial z} - \epsilon
$$

$$
\Delta \text{TKE} = \text{term 2} + \text{term 3} + \text{term 4} + \text{term 5} + \text{term 6}
$$
where

$$
\epsilon = \nu 
    \Big(
        \overline { 
            \frac{\partial \overline{u'}}{\partial x}^2
        }
    +
        \overline { 
            \frac{\partial \overline{v'}}{\partial x}^2
        }
    +
        \overline { 
            \frac{\partial \overline{w'}}{\partial x}^2
        }
    \Big)
$$
where:

term 1: temporal change in local TKE

term 2: bouyant production or destribution of TKE

term 3: redistribution of TKE by pressure fluctuations

term 4: shear/friction production of turbulence

term 5: turbulent transport of TKE

term 6: viscous dissipation of TKE